In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df_label = pd.read_csv("../input/dog-breed-identification/labels.csv")
df_label.head()

In [ ]:
df_label.shape
df_label['breed'].value_counts()

In [ ]:
label_name = df_label['breed'].sort_values().unique()

In [ ]:
df_label.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_label['breed'])
df_label['breed'] = encoder.transform(df_label['breed'])


In [ ]:
encoder.inverse_transform([19,37])

In [ ]:
df_label['id'] = df_label['id'] + '.jpg'
df_label.head()

In [ ]:
df_label['breed'] = df_label['breed'].astype(str)
df_label.info()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator ( 
    rescale =1/255.,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range =0.1,
    shear_range = 0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode = 'nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator ( 
    rescale=1/255. ,
    validation_split=0.2
)

In [ ]:
DATASET_DIR = '/kaggle/input/dog-breed-identification'
train_dir = '/kaggle/input/dog-breed-identification/train'
test_dir = '/kaggle/input/dog-breed-identification/test'

In [ ]:
train_generator = train_datagen.flow_from_dataframe (
    dataframe = df_label,
    directory = train_dir,
    x_col = 'id' ,     
    y_col = 'breed',      
    target_size = (150,150),
    batch_size = 128,
    class_mode = 'categorical',
    subset = 'training' ,
    seed = 7   
)
val_generator = val_datagen.flow_from_dataframe (
    dataframe = df_label,
    directory = train_dir,
    x_col = 'id' ,     
    y_col = 'breed',      
    target_size = (150,150),
    batch_size = 128,
    class_mode = 'categorical',
    subset = 'validation' ,
    seed = 7   
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models ,optimizers
model = tf.keras.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
history = model.fit(train_generator,
                    epochs=1,
                    steps_per_epoch=len(train_generator)-1,
                    validation_data=val_generator,
                    validation_steps=len(val_generator)-1
                   )

In [ ]:
base_dir = '../input/dog-breed-identification'
test_datagen = ImageDataGenerator (
rescale = 1/255,
)

test_generator = test_datagen.flow_from_directory (
    base_dir,
    target_size = (150,150),
    batch_size = 128,
    classes=['test']
)


In [ ]:
preds = model.predict(test_generator,batch_size=len(test_generator)-1)

In [ ]:
preds.shape

In [ ]:
df_submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
df

In [ ]:
class_to_num = dict(zip(label_name, range(120)))  
for name in label_name:  
    df_submission[name] = preds[:,class_to_num[name]]


In [ ]:
df_submission[name].head()

In [ ]:
df_submission.to_csv('submission_dog_cnn.csv',index=False)